# Secure Boosting Tree

## 資料品質檢查

### 設定資料路徑 & 參數

In [1]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "motor_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/motor_hetero_guest.csv")

### 缺失值 & 欄位名

In [2]:
import pandas as pd
dense_df = pd.read_csv(dense_data_dir)
print(dense_df.isna().sum())
print(dense_df.head(5))

idx               0
motor_speed       0
pm                0
stator_yoke       0
stator_tooth      0
stator_winding    0
dtype: int64
   idx  motor_speed        pm  stator_yoke  stator_tooth  stator_winding
0    1     0.293536 -0.633105    -0.397535     -0.290147       -0.307720
1    2    -1.222430 -0.150656    -0.777613     -1.226843       -1.321856
2    3    -0.951901  1.136038     0.967011      0.454562        0.110899
3    4     1.503905  1.046629     1.291209      1.302569        1.049111
4    5     0.186439 -0.470173    -0.053767     -0.123047       -0.032705


## 上傳資料

In [3]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2023-06-07 06:01:52.244 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070601519561680

2023-06-07 06:01:52.251 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-06-07 06:01:53.267 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:01:53.268 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-06-07 06:01:54.283 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-07 06:01:55.298 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-07 06:01:56.313 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-07 06:01:57.327 | INFO     

### 建構 Training pipeline 範例

使用 `pipeline` 模塊來構建聯邦學習流程

In [4]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

實例化 `pipeline` 並設定 `initiator` 和 `roles`:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [5]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=guest) \
        .set_roles(guest=guest, host=host)

使用 `Reader` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 Reader`

In [6]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "motor_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "motor_hetero_host", "namespace": "experiment"})

使用 `DataTransform` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 DataTransform`

`DataTransform` 負責資料前處理( 設定目標欄位名稱, 補缺值, 替換 outliers )

In [7]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name='motor_speed', label_type='float', 
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

使用 `Intersection` 模塊來達到對齊雙方相同顧客 ID, 且不會洩漏非相同顧客的 ID

In [8]:
intersect_0 = Intersection(name="intersect_0")

現在使用 `HeteroSecureBoost` 模塊. 用以下的參數來構建樹模型

In [9]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="regression",
                                         objective_param={"objective": "lse"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


最後, 為了檢驗好壞 使用 `Evaluation` 模塊來驗證好壞

In [10]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="regression")

上一個 component 的 output 是下一個 component 的 input

    - data_transform_0 吃 reader_0 的 output
    - intersect_0 吃 data_transform_0 的 output
    - hetero_secureboost_0 吃 intersect_0 的 output
    - evaluation_0 吃 hetero_secureboost_0 的 output (預測值)

記得用 `pipeline.compile()` 來打包整串流程
使用 `pipeline.fit()` 來開始進行訓練

In [11]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-07 06:02:05.912 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070602014971610

2023-06-07 06:02:05.920 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-07 06:02:06.929 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-07 06:02:07.947 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:02:07.948 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-07 06:02:08.966 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-07 06:02:09.982 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-07 06:02:10.997 | INFO     | pipel

2023-06-07 06:02:43.527 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-06-07 06:02:44.546 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-07 06:02:45.574 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-07 06:02:46.598 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-07 06:02:47.613 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
m2023-06-07 06:02:48.631 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:02:48.633 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component in

2023-06-07 06:03:22.869 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:16
2023-06-07 06:03:23.885 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:17
m2023-06-07 06:03:24.907 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:03:24.909 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-06-07 06:03:25.925 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-06-07 06:03:26.943 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:21
2023-06-07 06:03:27.961 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component 

2023-06-07 06:04:00.325 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-06-07 06:04:01.341 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-06-07 06:04:02.356 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-06-07 06:04:03.371 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-06-07 06:04:04.395 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:58
2023-06-07 06:04:05.427 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-06-07 06:04:06.446 | INFO     | pip

2023-06-07 06:04:38.020 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:32
2023-06-07 06:04:39.037 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:33
2023-06-07 06:04:40.053 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:34
2023-06-07 06:04:41.083 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:35
2023-06-07 06:04:42.100 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:36
2023-06-07 06:04:43.115 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:37
2023-06-07 06:04:44.129 | INFO     | pip

...

2023-06-07 06:09:39.480 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:33
2023-06-07 06:09:40.501 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:34
2023-06-07 06:09:41.516 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:35
2023-06-07 06:09:42.532 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:36
2023-06-07 06:09:43.550 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:37
2023-06-07 06:09:44.567 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:38
2023-06-07 06:09:45.582 | INFO     | pip

2023-06-07 06:10:17.133 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:11
2023-06-07 06:10:18.156 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:12
2023-06-07 06:10:19.172 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:13
2023-06-07 06:10:20.189 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:14
2023-06-07 06:10:21.204 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:15
2023-06-07 06:10:22.220 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:16
2023-06-07 06:10:23.246 | INFO     | pip

2023-06-07 06:10:54.767 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:48
2023-06-07 06:10:55.783 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:49
2023-06-07 06:10:56.798 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:50
2023-06-07 06:10:57.814 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:51
2023-06-07 06:10:58.834 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:52
2023-06-07 06:10:59.850 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:53
2023-06-07 06:11:00.866 | INFO     | pip

2023-06-07 06:11:32.417 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:26
2023-06-07 06:11:33.433 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:27
2023-06-07 06:11:34.448 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:28
2023-06-07 06:11:35.467 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:29
2023-06-07 06:11:36.482 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:30
2023-06-07 06:11:37.498 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:31
2023-06-07 06:11:38.514 | INFO     | pip

2023-06-07 06:12:10.071 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:04
2023-06-07 06:12:11.086 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:05
2023-06-07 06:12:12.102 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:06
2023-06-07 06:12:13.118 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:07
2023-06-07 06:12:14.134 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:08
2023-06-07 06:12:15.159 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:10:09
2023-06-07 06:12:16.178 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_st

當訓練結束後, 模型會用來做預測. 使用者可以自由選擇要不要儲存此次 `pipeline` 以方便未來重複使用
使用 `pipeline.dump(pipeline_saved_path)` 來完成儲存

In [12]:
pipeline.dump("pipeline_saved/hetero_two_party_continual_input_regression_secure_boost_tree.pkl");

### 建構 Inference pipeline 範例

首先, 使用 `PipeLine.load_model_from_file` load `pkl` 檔

部署 Inference 需要的模塊, 在這邊是 `data_transform_0`, `intersect_0`, `hetero_secureboost_0`

In [13]:
pipeline = PipeLine.load_model_from_file('pipeline_saved/hetero_two_party_continual_input_regression_secure_boost_tree.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

接著, 部署 `Reader` 模塊 `reader_1` 來讀取新data

In [14]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=guest).component_param(table={"name": "motor_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=host).component_param(table={"name": "motor_hetero_host", "namespace": "experiment"})

最後, 部署新的 `Evaluation` 來衡量 predict ( Inference ) 的表現

In [15]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="regression")

整合所有模塊

In [16]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


預測!

In [17]:
predict_pipeline.predict()

2023-06-07 06:12:26.652 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306070612261097640

2023-06-07 06:12:26.660 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-07 06:12:27.669 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-07 06:12:28.684 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:12:28.685 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:02
2023-06-07 06:12:29.702 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:03
2023-06-07 06:12:30.717 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_1, time elapse: 0:00:04
2023-06-07 06:12:31.740 | INFO     | pipel

2023-06-07 06:13:03.728 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
2023-06-07 06:13:04.746 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-07 06:13:05.761 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-07 06:13:06.778 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-07 06:13:07.794 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
m2023-06-07 06:13:08.809 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:13:08.810 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component in

2023-06-07 06:13:42.544 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:15
m2023-06-07 06:13:44.595 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-07 06:13:44.597 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:17
2023-06-07 06:13:45.614 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-06-07 06:13:46.630 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:19
2023-06-07 06:13:47.644 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-06-07 06:13:48.663 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running c

2023-06-07 06:14:20.554 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:53
2023-06-07 06:14:21.576 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-06-07 06:14:22.613 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-06-07 06:14:23.630 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-06-07 06:14:24.645 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-06-07 06:14:25.661 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:59
2023-06-07 06:14:26.675 | INFO     | pip

用 `pipeline.get_component('evaluation_0').get_summary()` 

來取得 `evaluation_0` 模塊的資訓儲並存成 json 檔

In [18]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/hetero_two_party_continual_input_regression_secure_boost_tree.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/two_party_continual_input_regression.json 
 {
    "hetero_secureboost_0": {
        "train": {
            "explained_variance": 0.9226178534666176,
            "mean_absolute_error": 0.21740692326486935,
            "mean_squared_error": 0.07876890259778647,
            "median_absolute_error": 0.194147640835375,
            "r2_score": 0.922617852701248,
            "root_mean_squared_error": 0.2806579815323029
        }
    }
}


In [20]:
pipeline.get_component('hetero_secureboost_0').get_output_data(10)

,idx,label,predict_result,predict_score,predict_detail,type
0,300,-0.9518769,-0.773935681865375,-0.773935681865375,{'label': -0.773935681865375},train
1,510,0.45712125,0.5033330459946249,0.5033330459946249,{'label': 0.5033330459946249},train
2,511,-1.1899537,-0.9461662783853749,-0.9461662783853749,{'label': -0.9461662783853749},train
3,2,-1.22243,-0.9959933263553751,-0.9959933263553751,{'label': -0.9959933263553751},train
4,432,0.41346514,0.33720430517462496,0.33720430517462496,{'label': 0.33720430517462496},train
5,404,-0.90669906,-0.6531124519253749,-0.6531124519253749,{'label': -0.6531124519253749},train
6,556,-1.0680722,-0.846018224825375,-0.846018224825375,{'label': -0.846018224825375},train
7,211,0.57955635,0.675332198304625,0.675332198304625,{'label': 0.675332198304625},train
8,569,-0.77171844,-0.603088511315375,-0.603088511315375,{'label': -0.603088511315375},train


# Hetero_NN

In [1]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "motor_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/motor_hetero_guest.csv")

In [2]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2023-06-12 02:10:13.532 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306120210132099270

2023-06-12 02:10:13.539 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-12 02:10:14.548 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-12 02:10:15.562 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-12 02:10:15.563 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-12 02:10:16.578 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-12 02:10:17.592 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-12 02:10:18.608 | INFO     | pipel

In [6]:
import torch as t
from torch import nn
from torch import optim

from collections import OrderedDict
from pipeline import fate_torch as ft
from pipeline import fate_torch_hook
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import Evaluation
from pipeline.component import HeteroNN
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.interface import Data
from pipeline.interface import Model
from pipeline.utils.tools import load_job_config

# this is important, modify torch modules so that Sequential model be parsed by pipeline
fate_torch_hook(t)

pipeline = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "motor_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "motor_hetero_host", "namespace": "experiment"})

data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name='motor_speed', label_type='float', 
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=0.0,
    outlier_replace=False, outlier_replace_method=None, outlier_replace_value=0.0
)
intersection_0 = Intersection(name="intersection_0")

# define network structure in torch style #
# define guest model
Linear = nn.Linear
ReLU = nn.ReLU
guest_bottom_a = Linear(4, 8, True)
seq = nn.Sequential(
    OrderedDict([
        ('layer_0', guest_bottom_a),
        ('relu_0', ReLU())
    ])
)

seq2 = nn.Sequential(
    ReLU(),
    Linear(8, 1, True),
) 

# define host model
seq3 = nn.Sequential(
    Linear(7, 8, True),
    ReLU()
)

# use interactive layer after fate_torch_hook
interactive_layer = t.nn.InteractiveLayer(out_dim=8, guest_dim=8, host_dim=8, host_num=1)

# loss fun
mse_loss_fn = nn.MSELoss()

# optimizer, after fate torch hook optimizer can be created without parameters
opt: ft.optim.Adam = optim.Adam(lr=0.01)

hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=30, floating_point_precision=None,
                       interactive_layer_lr=0.1, batch_size=-1, early_stop="diff")
guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
guest_nn_0.add_bottom_model(seq)
guest_nn_0.add_top_model(seq2)
host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)
host_nn_0.add_bottom_model(seq3)

hetero_nn_0.set_interactive_layer(interactive_layer)

hetero_nn_0.compile(opt, loss=mse_loss_fn)

hetero_nn_1 = HeteroNN(name="hetero_nn_1")
evaluation_0 = Evaluation(name="evaluation_0", eval_type="regression")

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersection_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_nn_0, data=Data(train_data=intersection_0.output.data))
pipeline.add_component(hetero_nn_1, data=Data(test_data=intersection_0.output.data),
                       model=Model(model=hetero_nn_0.output.model))
pipeline.add_component(evaluation_0, data=Data(data=hetero_nn_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-12 02:43:01.626 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306120243012308400

2023-06-12 02:43:01.634 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-06-12 02:43:02.643 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-06-12 02:43:03.661 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-12 02:43:03.662 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-12 02:43:04.677 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-12 02:43:05.693 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-12 02:43:06.709 | INFO     | pipel

m2023-06-12 02:43:40.733 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-12 02:43:40.735 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:39
2023-06-12 02:43:41.749 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:40
2023-06-12 02:43:42.767 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:41
2023-06-12 02:43:43.783 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:42
2023-06-12 02:43:44.797 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:43
2023-06-12 02:43:45.811 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection

2023-06-12 02:44:19.306 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:17
2023-06-12 02:44:20.320 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:18
2023-06-12 02:44:21.334 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:19
2023-06-12 02:44:22.356 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:20
2023-06-12 02:44:23.411 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:21
2023-06-12 02:44:24.448 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:22
2023-06-12 02:44:25.468 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-12 02:44:59.172 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:57
2023-06-12 02:45:00.206 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:58
2023-06-12 02:45:01.224 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:59
2023-06-12 02:45:02.239 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:00
2023-06-12 02:45:03.255 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:01
2023-06-12 02:45:04.270 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:02
2023-06-12 02:45:05.286 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-12 02:45:38.927 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:37
2023-06-12 02:45:39.942 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:38
2023-06-12 02:45:40.957 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:39
2023-06-12 02:45:41.972 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:40
2023-06-12 02:45:42.992 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:41
2023-06-12 02:45:44.017 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:42
2023-06-12 02:45:45.033 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

...

2023-06-12 03:00:11.613 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:09
2023-06-12 03:00:12.630 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:11
2023-06-12 03:00:13.655 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:12
2023-06-12 03:00:14.670 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:13
2023-06-12 03:00:15.687 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:14
2023-06-12 03:00:16.702 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:15
2023-06-12 03:00:17.720 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-12 03:00:51.259 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:49
2023-06-12 03:00:52.280 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:50
2023-06-12 03:00:53.295 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:51
2023-06-12 03:00:54.311 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:52
2023-06-12 03:00:55.328 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:53
2023-06-12 03:00:56.344 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:54
2023-06-12 03:00:57.360 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-12 03:01:30.924 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:29
2023-06-12 03:01:31.939 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:30
2023-06-12 03:01:32.958 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:31
2023-06-12 03:01:33.973 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:32
2023-06-12 03:01:34.992 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:33
2023-06-12 03:01:36.008 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:34
2023-06-12 03:01:37.029 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-12 03:02:09.755 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 0:19:08
2023-06-12 03:02:10.770 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 0:19:09
2023-06-12 03:02:11.795 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 0:19:10
2023-06-12 03:02:12.812 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 0:19:11
2023-06-12 03:02:13.833 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 0:19:12
2023-06-12 03:02:14.849 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_1, time elapse: 0:19:13
2023-06-12 03:02:15.867 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-06-12 03:02:49.494 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:89 - Job is success!!! Job id is 202306120243012308400
2023-06-12 03:02:49.495 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:90 - Total time: 0:19:47


In [7]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/hetero_two_party_continual_input_regression_hetero_nn.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")

Write in metadata_saved_dir : /data/projects/fate/persistence/metadata/two_party_continual_input_regression_hetero_nn.json 
 {
    "hetero_nn_0": {
        "train": {
            "explained_variance": 0.8989518595511835,
            "mean_absolute_error": 0.2481708641606383,
            "mean_squared_error": 0.10376519412812403,
            "median_absolute_error": 0.1963573694229126,
            "r2_score": 0.8980616298858738,
            "root_mean_squared_error": 0.32212605316571963
        }
    }
}
